In [ ]:
import numpy as np
import torch
from torch.utils.data import Dataset
import os


class FlowDataset(Dataset):
    def __init__(self, num_points=16, data_dir='flow/', data_session='all_params_50625', mode='train'):
        data_fn = os.path.join(data_dir, 'dataset_' + data_session + '.npy')
        bc_fn = os.path.join(data_dir, 'bc_'+ data_session + '.npy')
        target = np.load(data_fn)
        inputs = np.load(bc_fn)

        # Pre-Process Data
        # replace first time step with all the boundary conditions
        # for now, hardcoded to fill in since there are 4 bc's and 17 points
        for i in range(len(inputs)):
            D = inputs[i][0]
            dpdx = inputs[i][1]
            mu = inputs[i][2]
            nu = inputs[i][3]
            for j in range(num_points+1):
                if 0 <= j < 5:
                    target[i][0][j] = D
                elif 5 <= j < 9:
                    target[i][0][j] = dpdx
                elif 9 <= j < 13:
                    target[i][0][j] = mu
                else:
                    target[i][0][j] = nu

        num_data = len(inputs)
        np.random.seed(0)
        # split the dataset inton training and test 
        test_idx = np.random.choice(num_data, num_data//5, replace=False).tolist()
        train_idx = list(set(range(num_data)) - set(test_idx))

        self.mode = mode
        if mode is 'train':
            self.data = target[train_idx,:,:].astype(np.float32)
        elif mode is 'test':
            self.data = target[test_idx,:,:].astype(np.float32)
    
    def __getitem__(self, idx):
        if self.mode is 'train':
            return self.data[idx,:-1,:], self.data[idx,1:,:]
        elif self.mode is 'test':
            return self.data[idx,0,:], self.data[idx,1:,:]
    
    def __len__(self):
        return len(self.data)


In [ ]:
import torch
import torch.nn as nn
from torch.autograd import Variable


class FlowLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, dropout,):
        super(FlowLSTM, self).__init__()
        
        #define variables
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.dropout = dropout

        self.lstm = [nn.LSTMCell(input_size=self.input_size,hidden_size=self.hidden_size)]
        
        for i in range(num_layers-1):
            self.lstm += [nn.LSTMCell(input_size=self.hidden_size,hidden_size=self.hidden_size)]
            
        self.drop_layer = nn.Dropout(p = self.dropout)
        self.linear = nn.Linear(self.hidden_size,self.input_size)
        

    def forward(self, x):
        '''
        input: x of dim (batch_size, 19, 17)
        '''
        outps = []
        
        h_t = torch.zeros(x.size(0),self.hidden_size,dtype=torch.float32)
        c_t = torch.zeros(x.size(0),self.hidden_size,dtype=torch.float32)
        h_t2 = torch.zeros(x.size(0),self.hidden_size,dtype=torch.float32)
        c_t2 = torch.zeros(x.size(0),self.hidden_size,dtype=torch.float32)
        for _,sample in enumerate(x.split(1,dim=1)):
            for i in range(self.num_layers):
                if i ==0:
                    h_t,c_t = self.lstm[i](sample.reshape(sample.shape[0],-1),(h_t,c_t))
                else:
                    h_t2,c_t2 = self.lstm[i](h_t,(h_t2,c_t2))
            
            outp = self.drop_layer(h_t2)
            outp = self.linear(outp)
            outp = outp.reshape(outp.shape[0],1,-1)
            outps += [outp]
        outps = torch.cat(outps,dim=1)
        return outps

    def test(self, x):
        '''
        input: x of dim (batch_size, 17)
        '''
        outps = []
        out = x
        
        h_t = torch.zeros(x.size(0),self.hidden_size,dtype=torch.float32)
        c_t = torch.zeros(x.size(0),self.hidden_size,dtype=torch.float32)
        h_t2 = torch.zeros(x.size(0),self.hidden_size,dtype=torch.float32)
        c_t2 = torch.zeros(x.size(0),self.hidden_size,dtype=torch.float32)
        
        for i in range (19):
            for i in range(self.num_layers):
                if i ==0:
                    h_t,c_t = self.lstm[i](out,(h_t,c_t))
                else:
                    h_2t,c_2t = self.lstm[i](h_t,(h_t2,c_t2))
            out = self.linear(h_2t)
            outps += [out.reshape(out.shape[0],1,-1)]
        outps = torch.cat(outps,dim=1)

        return outps

In [ ]:
import os
import numpy as np
import torch
from torch import nn
from torch.optim import Adam
from torch.utils.data import DataLoader




def main():
    # check if cuda available
    Epoch_List = []
    Loss_List = []
    device = 'cuda:0' if torch.cuda.is_available() else 'cpu'

    # define dataset and dataloader
    train_dataset = FlowDataset(mode='train').data
    test_dataset = FlowDataset(mode='test').data
    train_loader = DataLoader(dataset=train_dataset, batch_size=16, shuffle=True, num_workers=4)
    test_loader = DataLoader(dataset=test_dataset, batch_size=16, shuffle=False, num_workers=4)

    # hyper-parameters
    num_epochs = 20
    lr = 0.001
    input_size = 17 # do not change input size
    hidden_size = 256
    num_layers = 2
    dropout = 0.1

    model = FlowLSTM(
        input_size=input_size, 
        hidden_size=hidden_size, 
        num_layers=num_layers, 
        dropout=dropout
    ).to(device)

    # define your LSTM loss function here
    criterion = nn.MSELoss()
    # define optimizer for lstm model
    optim = Adam(model.parameters(), lr=lr)
    #model 
    for p in model.parameters():
        print(p.numel())
    print(model)
    for epoch in range(num_epochs):
        for n_batch, data in enumerate(train_loader):
            in_batch = data[:,:-1,:]
            label = data[:,1:,:]
            in_batch, label = in_batch.to(device), label.to(device)
            out = model.forward(in_batch)
            loss = criterion(out, label)
            # train LSTM
            # calculate LSTM loss
            # loss = loss_func(...)

            optim.zero_grad()
            with torch.autograd.set_detect_anomaly(True):
                loss.backward()
            optim.step()

            # print loss while training

            if (n_batch + 1) % 200 == 0:
                print("Epoch: [{}/{}], Batch: {}, Loss: {}".format(
                    epoch, num_epochs, n_batch, loss.item()))
                if n_batch == 199:
                    Epoch_List.append(epoch)
                    Loss_List.append(loss.item())
    # test trained LSTM model
    l1_err, l2_err = 0, 0
    l1_loss = nn.L1Loss()
    l2_loss = nn.MSELoss()
    model.eval()
    with torch.no_grad():
        for n_batch, data in enumerate(test_loader):
            in_batch = data[:,0,:]
            label = data[:,1:,:]
            in_batch, label = in_batch.to(device), label.to(device)
            pred = model.test(in_batch)

            l1_err += l1_loss(pred, label).item()
            l2_err += l2_loss(pred, label).item()

    print("Test L1 error:", l1_err)
    print("Test L2 error:", l2_err)
    torch.save(model.state_dict(),'p1_model1.ckpt')
    from google.colab import drive
    drive.mount('/content/gdrive')
    model_save_name = 'p1_model1.ckpt'
    path = F"/content/gdrive/MyDrive/{model_save_name}"
    torch.save(model.state_dict(),path)

    # visualize the prediction comparing to the ground truth
    if device is 'cpu':
        pred = pred.detach().numpy()[0,:,:]
        label = label.detach().numpy()[0,:,:]
    else:
        pred = pred.detach().cpu().numpy()[0,:,:]
        label = label.detach().cpu().numpy()[0,:,:]

    r = []
    num_points = 17
    interval = 1./num_points
    x = int(num_points/2)
    for j in range(-x,x+1):
        r.append(interval*j)

    from matplotlib import pyplot as plt
    plt.figure()
    for i in range(1, len(pred)):
        c = (i/(num_points+1), 1-i/(num_points+1), 0.5)
        plt.plot(pred[i], r, label='t = %s' %(i), c=c)
    plt.xlabel('velocity [m/s]')
    plt.ylabel('r [m]')
    plt.legend(bbox_to_anchor=(1,1),fontsize='x-small')
    plt.show()

    plt.figure()
    for i in range(1, len(label)):
        c = (i/(num_points+1), 1-i/(num_points+1), 0.5)
        plt.plot(label[i], r, label='t = %s' %(i), c=c)
    plt.xlabel('velocity [m/s]')
    plt.ylabel('r [m]')
    plt.legend(bbox_to_anchor=(1,1),fontsize='x-small')
    plt.show()

    plt.figure()
    plt.plot(Epoch_List,Loss_List)
    plt.title('Epoch vs Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.show()

if __name__ == "__main__":
    main()